In [ ]:
# Creates various plots for the accuracy of various approaches.

In [ ]:
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models

from tqdm import tqdm
from pathlib import Path
from PIL import Image

from arid import arid
import json

def map_boxes(box):
    x1, y1, x2, y2 = box
    return ((x1, y1), (x2, y1), (x2, y2), (x1, y2))

wps = arid.get_wps("/home/justin/Desktop/arid-dataset")
ss_root = Path('/home/justin/Desktop/thesis/ss')

_wps = {_wp.get_title(): _wp for _wp in wps}

In [ ]:
target_objects = {
    'apple_0': 'green apple fruit',
    'apple_1': 'yellow apple fruit',
    'apple_2': 'red apple fruit',
    'banana_0': 'green banana', 
    'banana_1': 'yellow banana',
    'banana_2': 'yellow banana',
    'bell_pepper_0': 'green bell pepper', 
    'bell_pepper_1': 'red bell pepper',
    'bell_pepper_2': 'yellow bell pepper',
    'bowl_0': 'white bowl',
    'bowl_1': 'blue bowl',
    'bowl_2': 'white bowl',
    'cell_phone_0': 'black smartphone',
    'cell_phone_1': 'white smartphone',
    'cell_phone_2': 'black white smartphone',
    'coffee_mug_0': 'blue coffee mug',
    'coffee_mug_1': 'black white coffee mug',
    'coffee_mug_2': 'white coffee mug',
    'keyboard_0': 'black computer keyboard',
    'keyboard_1': 'white computer keyboard',
    'keyboard_2': 'black computer keyboard',
    'orange_0': 'orange fruit',
    'orange_1': 'orange fruit',
    'orange_2': 'orange fruit',
    'scissors_0': 'green handle scissors',
    'scissors_1': 'purple handle scissors',
    'scissors_2': 'black handle scissors',
    'toothbrush_0': 'toothbrush',
    'toothbrush_2': 'green white toothbrush',
    'water_bottle_0': 'empty water bottle',
    'water_bottle_1': 'empty water bottle',
    'water_bottle_2': 'empty water bottle',
    'ball_0': 'blue ball',
    'binder_0': 'green three ring binder',
    'calculator_0': 'ti calculator',
    'camera_0': 'silver digital camera',
    'cap_2': 'black baseball hat',
    'cereal_box_0': 'corn flakes cereal box',
    'comb_0': 'brown hair comb',
    'dry_battery_0': 'double a duracell battery',
    'flashlight_2': 'black flashlight',
    'greens_0': 'leafy green vegetable',
    'lemon_0': 'yellow lemon',
    'lightbulb_0': 'white lightbulb',
    'mushroom_0': 'white mushroom',
    'pliers_1': 'needlenose pliers',
    'shampoo_0': 'grey shampoo bottle',
    'toothpaste_0': 'toothpaste tube',
    'stapler_0': 'grey stapler',
    'potato_0': 'potato',
    'sponge_0': 'yellow sponge',
    'tomato_0': 'red tomato',
}
print(len(target_objects))

In [ ]:
# FOR OUR APPROACH

In [ ]:
CUTOFF = 100

EZ = 0.1
MED = 0.5

results = {
    'top_one': {},
    'top_five': {}
}

for OBJ in tqdm(target_objects):
    
    target_wps = {}
    for wp in wps:
        imgs = wp.get_images_with_object(OBJ)
        if imgs:
            target_wps[wp.get_title()] = list(map(lambda x: x['path'].stem, imgs))

    results_top_one_ez = 0
    results_top_one_med = 0
    results_top_five_ez = 0
    results_top_five_med = 0
    total = 0

    for title, img_ids in target_wps.items():
        wp = _wps[title]
        data = Path(f'/home/justin/Desktop/thesis-results/{OBJ}-{title}.json')
        with open(data) as f:
            rank_data = json.load(f)

        ss_path = Path(ss_root /'ss-fast'/ f'ss-fast-{title}.json')
        with open(ss_path) as f:
            ss_data = json.load(f)

        img_paths = wp.rgb_image_paths()

        for img_id in img_ids:

            # FOR DISPLAY.
    #         actual_img = None
    #         for img_path in img_paths:
    #             if img_path.stem == img_id:
    #                 actual_img = Image.open(img_path)

            # GET ANNOTATIONS
            annotations = []
            display_annotations = []
            for annotation in wp.get_annotations(img_id)['annotations']:
                if annotation['id'] == OBJ:
                    x1 = annotation['x']
                    y1 = annotation['y']
                    x2 = x1 + annotation['width']
                    y2 = y1 + annotation['height']
                    annotations.append(((x1, y1), (x2, y1), (x2, y2), (x1, y2)))

    #                 display_annotations.append({
    #                     'id': f'_',
    #                     'coords': ((x1, y1), (x2, y1), (x2, y2), (x1, y2)),
    #                     'score': 0.99,
    #                     'colormap': 'winter'
    #                 })

            # GET SS BOXES
            ss_boxes = ss_data[img_id]['boxes']
            scores = rank_data[img_id]
            ss_scores = sorted(list(zip(ss_boxes, scores)), key=lambda s:s[1], reverse=True)
            hi_ss_scores = ss_scores[:CUTOFF]
            taken = [False] * CUTOFF # used for non-max suppression

            # GET TOP 5 PREDICTIONS
            top_five = []
            display_top_five = []
            for idx, (box, score) in enumerate(hi_ss_scores):
                x1, y1, x2, y2 = box
                coords = map_boxes(box)

                # Step through lower scored predictions and remove overlapping ones.
                for jdx in range(idx+1, len(hi_ss_scores)):
                    iou = arid.compute_bbox_iou(coords, map_boxes(hi_ss_scores[jdx][0]))
                    if iou > 0.09:
                        taken[jdx] = True

                if not taken[idx]:
                    top_five.append(coords)
    #                 display_top_five.append({
    #                     'id': f'{str(idx)}',
    #                     'coords': (coords),
    #                     'score': 0.99 - (idx * 0.05) ,
    #                     'colormap': 'spring_r'
    #                 })
                    if len(top_five) == 5:
                        break
                
            if len(annotations) == 0:
                print(f'{title}-{img_id}')

            for annotation in annotations:
                total += 1
                
                ez_found = False
                med_found = False

                for idx, top in enumerate(top_five):
                    iou = arid.compute_bbox_iou(top, annotation)
                    # TODO: this is ugly logic. We're trying to ensure
                    if iou >= MED and not med_found:
                        results_top_five_ez += 1
                        results_top_five_med += 1

                        if idx == 0:
                            results_top_one_ez += 1
                            results_top_one_med += 1                        
                        ez_found = True
                        med_found = True

                    elif iou >= EZ and not ez_found:
                        results_top_five_ez += 1

                        if idx == 0:
                            results_top_one_ez += 1
                        
                        ez_found = True



    #         display_annotations.extend(display_top_five)
    #         arid.annotate_img(actual_img, 'test', display_annotations, save=False)
    #         display(actual_img)

    if total == 0:
        print(OBJ)
    else:
        results_top_one_ez = results_top_one_ez / total
        results_top_one_med = results_top_one_med / total
        results_top_five_ez = results_top_five_ez / total
        results_top_five_med = results_top_five_med / total

        results['top_one'][OBJ] = {
            'ez': results_top_one_ez,
            'med': results_top_one_med,
        }
        results['top_five'][OBJ] = {
            'ez': results_top_five_ez,
            'med': results_top_five_med,
        }
    
with open('final-results.json', 'w') as outfile:
    json.dump(results, outfile)

In [ ]:
# Create plot for the results
def plot_results(scores, f_name, w=10, t=90):
    import numpy as np
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(20,28))
    font = {'family' : 'DejaVu Sans',
        'weight' : 'regular',
        'size'   : 22}

    plt.rc('font', **font)
    
    s5 = [v['ez']*100 for k, v in scores.items()]
    s7 = [v['med']*100 for k, v in scores.items()]

    ind = np.arange(len(scores))    # the x locations for the groups
    width = 0.25         # the width of the bars
    a = plt.barh(ind + width, s5, height=width, label='0.1')
    b = plt.barh(ind, s7, height=width, label='0.5')

    plt.xlabel('Percent of Object Instances')
    plt.ylabel('Object Name')
    plt.yticks(ind, scores.keys(), ha='right')
    plt.legend((a, b), ('0.1', '0.5'))
    plt.tight_layout()
    plt.savefig(f_name)


In [ ]:
with open('final-results.json') as outfile:
    scores = json.load(outfile)
    
one_score = {k: v for k, v in sorted(scores['top_one'].items(), key=lambda item: (item[1]['ez'], item[1]['med']))}
five_score = {k: v for k, v in sorted(scores['top_five'].items(), key=lambda item: (item[1]['ez'], item[1]['med']))}

In [ ]:
plot_results(one_score, 'final-prediction-top-one.png')
plot_results(five_score, 'final-prediction-top-five.png')

In [ ]:
print(len(one_score))

In [ ]:
# TO COMPARE TO FASTER R-CNN

In [ ]:
# Load wps and images that contain max one item from each overlapping object category for comparison.
targets = {}

mapping = {
    'apple_0': 'apple',
    'apple_1': 'apple',
    'apple_2': 'apple',
    'banana_0': 'banana',
    'banana_1': 'banana',
    'banana_2': 'banana',
    'orange_0': 'orange',
    'orange_1': 'orange',
    'orange_2': 'orange',
    'cell_phone_0': 'cell phone',
    'cell_phone_1': 'cell phone',
    'cell_phone_2': 'cell phone',
    'coffee_mug_0': 'cup',
    'coffee_mug_1': 'cup',
    'coffee_mug_2': 'cup',
    'scissors_0': 'scissors',
    'scissors_1': 'scissors',
    'scissors_2': 'scissors',
    'toothbrush_0': 'toothbrush',
    'toothbrush_2': 'toothbrush',
    'keyboard_0': 'keyboard',
    'keyboard_1': 'keyboard',
    'keyboard_2': 'keyboard',
    'bowl_0': 'bowl',
    'bowl_1': 'bowl', 
    'bowl_2': 'bowl',
    'water_bottle_0': 'bottle',
    'water_bottle_1': 'bottle', 
    'water_bottle_2': 'bottle', 
    
}

r_mapping = {
    'apple': 'apple',
    'banana': 'banana',
    'orange': 'orange',
    'cell phone': 'cell_phone',
    'cup': 'coffee_mug',
    'scissors': 'scissors',
    'toothbrush': 'toothbrush',
    'keyboard': 'keyboard',
    'bowl': 'bowl',
    'bottle': 'water_bottle',
}
itms = 0
target_objects = list(mapping.keys())
for wp in wps:
    title = wp.get_title()
    img_paths = wp.rgb_image_paths()
    for img_path in img_paths:
        img_id = img_path.stem
        for ann in wp.get_annotations(img_path.stem)['annotations']:
            if ann['id'] in target_objects:
                obj = ann['id']
                if targets.get(title) is None:
                    targets[title] = {}
                    
                if targets[title].get(img_id) is None:
                    targets[title][img_id] = []
                    
                obj_cat = '_'.join(obj.split('_')[:-1])
                obj_cats = list(map(lambda x: '_'.join(x.split('_')[:-1]),targets[title][img_id]))
                    
                if obj in targets[title][img_id]:
                    print(f'{title}-{img_id}-{obj}')
                elif obj_cat in obj_cats:
                    print(f'{title}-{img_id}-{obj_cat}')
                else:
                    itms += 1
                    targets[title][img_id].append(obj)
print(targets)

In [ ]:
DISPLAY = True
TO_DISPLAY = ['apple_1']

EZ = 0.1
MED = 0.5

thesis_results_root = Path(f'/home/justin/Desktop/thesis-results/')
arid_scene_root = Path('/home/justin/Desktop/arid-floyd')

# Load Faster R-CNN annotations.
data = Path(thesis_results_root  / 'faster-scores.json')
with open(data) as f:
    faster_annotations = json.load(f)


results = {}

for title, img_data in tqdm(targets.items()):
    wp = _wps[title]
    
    ss_path = Path(ss_root /'ss-fast'/ f'ss-fast-{title}.json')
    with open(ss_path) as f:
        ss_data = json.load(f)
    
    
    for img_id, target_object_instances in img_data.items():
        img_path = Path(arid_scene_root / title / f'{img_id}.png')
        img = Image.open(img_path)
        
        
        for target_object_instance in target_object_instances:
            
            # Initialize results for this object
            if results.get(target_object_instance) is None:
                results[target_object_instance] = {
                    'ours_ez': 0,
                    'ours_med': 0,
                    'faster_ez': 0,
                    'faster_med': 0,
                    'total': 0,
                }
            
            
            target_object_instance_category = '_'.join(target_object_instance.split('_')[:-1])

            # GET OUR PREDICTION
            data = Path(f'/home/justin/Desktop/thesis-results/{target_object_instance}-{title}.json')
            with open(data) as f:
                rank_data = json.load(f)
                
            ss_boxes = ss_data[img_id]['boxes']
            scores = rank_data[img_id]
            ss_scores = sorted(list(zip(ss_boxes, scores)), key=lambda s:s[1], reverse=True)
            
            displayed_predictions = []

            our_prediction = None
            for idx, (box, score) in enumerate(ss_scores):
                our_prediction = {
                    'id': f'ours',
                    'coords': map_boxes(box) ,
                    'score': 0.99 - (idx * 0.05) ,
                    'colormap': 'spring_r'
                }
                displayed_predictions.append(our_prediction)
                break
               
                
            # Faster Prediction (May be null)
            faster_prediction = None
            for fpred in faster_annotations[title][img_id]:
                if fpred['id'] == target_object_instance_category:
                    faster_prediction = fpred
                    displayed_predictions.append(faster_prediction)
                    break

            # GET GROUND TRUTH ANNOTATION
            annotation = {}
            for annotation in wp.get_annotations(img_id)['annotations']:
                if annotation['id'] == target_object_instance:
                    x1 = annotation['x']
                    y1 = annotation['y']
                    x2 = x1 + annotation['width']
                    y2 = y1 + annotation['height']
                    annotation = {
                        'id': f'ground',
                        'coords': ((x1, y1), (x2, y1), (x2, y2), (x1, y2)),
                        'score': 0.99,
                        'colormap': 'winter'
                    }
                    displayed_predictions.append(annotation)
                    break
                    
                    

            results[target_object_instance]['total'] += 1
            

            # our prediction
            iou = arid.compute_bbox_iou(our_prediction['coords'], annotation['coords'])
            if iou >= MED:
                results[target_object_instance]['ours_med'] += 1
            if iou >= EZ:
                results[target_object_instance]['ours_ez'] += 1
                
                
            if faster_prediction is not None:
                iou = arid.compute_bbox_iou(faster_prediction['coords'], annotation['coords'])
                if iou >= MED:
                    results[target_object_instance]['faster_med'] += 1
                if iou >= EZ:
                    results[target_object_instance]['faster_ez'] += 1

        if DISPLAY and target_object_instance in TO_DISPLAY:
            img1 = img.copy()
            arid.annotate_img(img1, 'test', displayed_predictions, save=False)
            display(img1)
            
            
final_results = {}

for object_instance, stats in results.items():
    t = stats['total']
    final_results[object_instance] = {
        'ours_ez': stats['ours_ez'] / t,
        'ours_med': stats['ours_med'] / t,
        'faster_ez': stats['faster_ez'] / t,
        'faster_med': stats['faster_med'] / t,
        'total': t,
    }
    
    
# final_results = {k:v for k, v in sorted(final_results.items(), key=lambda x: x[0])}
    
# with open('comparison_results.json', 'w') as outfile:
#     json.dump(final_results, outfile)


In [ ]:
# Create plot for the results
def plot_comparison_results(scores, f_name, w=10, t=90):
    import numpy as np
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(25,15))
    font = {'family' : 'DejaVu Sans',
        'weight' : 'regular',
        'size'   : 24}

    plt.rc('font', **font)
    
    me = [v['ours_ez']*100 for k, v in scores.items()]
#     mm = [v['ours_med']*100 for k, v in scores.items()]
    fe = [v['faster_ez']*100 for k, v in scores.items()]
#     fm = [v['faster_med']*100 for k, v in scores.items()]

    ind = np.arange(len(scores))    # the x locations for the groups
    width = 0.25       # the width of the bars
    a = plt.bar(ind, me, width, bottom=0, label='0.1 (ours)')
#     b = plt.bar(ind + width, mm, width, bottom=0, label='0.5 (ours)')
    d = plt.bar(ind + width, fe, width, bottom=0, label='0.1 (Faster R-CNN)')
#     e = plt.bar(ind + 3*width, fm, width, bottom=0, label='0.5 (Faster R-CNN)')

    plt.plot([2.5, 2.5], [0, 110], 'g--', lw=2)
    plt.plot([5.5, 5.5], [0, 110], 'g--', lw=2)
    plt.plot([8.5, 8.5], [0, 110], 'g--', lw=2)
    plt.plot([11.5, 11.5], [0, 110], 'g--', lw=2)
    plt.plot([14.5, 14.5], [0, 110], 'g--', lw=2)
    plt.plot([17.5, 17.5], [0, 110], 'g--', lw=2)
    plt.plot([20.5, 20.5], [0, 110], 'g--', lw=2)
    plt.plot([23.5, 23.5], [0, 110], 'g--', lw=2)
    plt.plot([25.5, 25.5], [0, 110], 'g--', lw=2)


    plt.ylabel('Percent of Object Instances')
    plt.ylim(top=100)
    plt.xlabel('Object Name')
    plt.xticks(ind, scores.keys(), rotation='45', ha='right')
    plt.legend((a, d), ('0.1 (ours)', '0.1 (Faster R-CNN)'), loc=(0.03, 0.87))
    plt.tight_layout()
    plt.savefig(f_name)


In [ ]:
with open('comparison_results.json') as outfile:
    results = json.load(outfile)
    

plot_comparison_results(results, 'approach_comparison.png')